# Agent-based model of tumor growth 

Author: [Peter McHale](http://ccbs-76.bio.uci.edu/~petermchale/)

This notebook lives at [Github](https://github.com/petermchale/tumor). Follow [these instructions](http://jupyter.readthedocs.io/en/latest/install.html)
to execute this Notebook using Python. Otherwise, [view this Notebook](https://nbviewer.jupyter.org/github/petermchale/tumor/blob/master/demonstration.ipynb?flush_cache=true) (and its inline animations) in read-only mode using [Jupyter Notebook Viewer](http://nbviewer.jupyter.org). 

## Introduction

## Making Python modules visible to the notebook

In [1]:
import os
repository_directory = os.getcwd() 
package_directory = repository_directory + '/tumor_package'
print('package directory:\t' + package_directory)
import sys
print('Python searches these paths when asked to import a module:')
sys.path.append(package_directory)
for path in sys.path: 
    print(path)

package directory:	/Users/ccbs/Desktop/tumor/tumor_package
Python searches these paths when asked to import a module:

/Users/ccbs/anaconda/lib/python35.zip
/Users/ccbs/anaconda/lib/python3.5
/Users/ccbs/anaconda/lib/python3.5/plat-darwin
/Users/ccbs/anaconda/lib/python3.5/lib-dynload
/Users/ccbs/anaconda/lib/python3.5/site-packages
/Users/ccbs/anaconda/lib/python3.5/site-packages/Sphinx-1.4.1-py3.5.egg
/Users/ccbs/anaconda/lib/python3.5/site-packages/aeosa
/Users/ccbs/anaconda/lib/python3.5/site-packages/setuptools-23.0.0-py3.5.egg
/Users/ccbs/anaconda/lib/python3.5/site-packages/IPython/extensions
/Users/ccbs/.ipython
/Users/ccbs/Desktop/tumor/tumor_package


## Simulating tumor growth 

In [3]:
# Initialize a virtual tumor
from simulation import animate_tumor_growth_base
from read import read_into_dict
initialCondition = read_into_dict('data/initialCondition.in')
parameterValues = read_into_dict('data/parameterValues.in')
fig, anim = animate_tumor_growth_base(initialCondition, parameterValues, 400, 2)

On Mac OS X, install [Homebrew](http://brew.sh) and use it to install [ffmpeg](https://ffmpeg.org) by typing `brew install ffmpeg` in the `Terminal`.

Embed the matplotlib animation in the notebook (follows [Louis Tiao's blog post](http://louistiao.me/posts/notebooks/embedding-matplotlib-animations-in-jupyter-notebooks/)): 

In [5]:
import matplotlib as mpl
print('matplotlib version:\t' + mpl.__version__ + '\t (should be at least 1.5.1)')
from matplotlib import pyplot as plt
print('animation writer:\t' + plt.rcParams['animation.writer'] + '\t (should be ffmpeg)')
from IPython.display import HTML
HTML(anim.to_html5_video())

matplotlib version:	1.5.1	 (should be at least 1.5.1)
animation writer:	ffmpeg	 (should be ffmpeg)


Let's suppose that the numbers of quiescent cells (Q-cells) feed-back both positively and negatively on the self-renewal probability of cycling cells (C-cells). At each cell cycle, on average, a C-cell (green) chooses to divide or become quiescent (red) with a probability $p$ or $1-p$, respectively, that depends upon the number of Q-cells within the local neighborhood. 

Each simulation begins with 1 C-cell and 3 Q-cells. Star indicates that the cell is mutant.

Point: Negative feedback can arrest tumor growth even though the spatial range of negative feedback is smaller than the maximum tumor size. 

Point: A front of C-cells transiently elongates the tumor.